In [1]:
import pandas as pd
import numpy as np

In [3]:
!pip install msgpack
!pip install beautifulsoup4

In [4]:
import requests
from bs4 import BeautifulSoup

Define URL

In [5]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

Capture table from URL

In [6]:
response = requests.get(url)
html = response.content
soup = BeautifulSoup(html, "lxml")
table = soup.find('tbody')

Create list from html table

In [7]:
list_of_rows = []
list_of_cells = []
app_row = True
last_zip = ""
same_zip = False
body = False

for row in table.findAll('tr'):
    col = 0
    borough = ""

    for cell in row.findAll('th'):
        text = cell.text.replace('&nbsp;', '')
        text = cell.text.replace('\n', '')
        list_of_cells.append(text)

    for cell in row.findAll('td'):
        text = cell.text.replace('&nbsp;', '')
        text = cell.text.replace('\n', '')
        col = col+1
        #print(str(col) + ": [" + text + "]")
        if (col==1):
            if (text==last_zip):
                same_zip = True
            else:
                if (app_row):
                    if (body):
                        list_of_cells.append(hood)
                    else:
                        body = True
                    #print(list_of_cells)
                    list_of_rows.append(list_of_cells)
                    #print("appended.")
                list_of_cells = []
                same_zip = False
                last_zip = text
                app_row = True
                list_of_cells.append(text)
                
        elif (col==2):
            
            borough = text
            if (text=="Not assigned"):
                app_row = False
            else:
                if (not same_zip):
                    list_of_cells.append(borough)
                    
            #print("    borough = " + borough)
            
        elif (col==3):
            
            if (app_row):
                
                if (text=="Not assigned"):
                    text = borough
                    
                if (same_zip):
                    hood = text + ", " + hood
                else:
                    hood = text
                
            #print("    text = " + text)
        #list_of_cells.append(text)
        
        #print(list_of_cells)


Create DataFrame from list with first row as header

Show # of rows in DataFrame

In [10]:
df = pd.DataFrame.from_records(list_of_rows[1:], columns=list_of_rows[0])
print(df.head(10))
df.shape

  Postcode           Borough                     Neighbourhood
0      M3A        North York                         Parkwoods
1      M4A        North York                  Victoria Village
2      M5A  Downtown Toronto         Regent Park, Harbourfront
3      M6A        North York  Lawrence Manor, Lawrence Heights
4      M7A      Queen's Park                      Queen's Park
5      M9A         Etobicoke                  Islington Avenue
6      M1B       Scarborough                    Malvern, Rouge
7      M3B        North York                   Don Mills North
8      M4B         East York   Parkview Hill, Woodbine Gardens
9      M5B  Downtown Toronto          Garden District, Ryerson


(103, 3)

Merge zip DataFrame with coordinate file to make Lat/Lon DataFrame

In [12]:
gc_df = pd.read_csv('Geospatial_Coordinates.csv')
gc_df.columns = ['Postcode', 'Latitude', 'Longitude']

latlon_df = pd.merge(df, gc_df, on='Postcode')
latlon_df.head(10)

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Queen's Park,Queen's Park,43.662301,-79.389494
5,M9A,Etobicoke,Islington Avenue,43.667856,-79.532242
6,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
7,M3B,North York,Don Mills North,43.745906,-79.352188
8,M4B,East York,"Parkview Hill, Woodbine Gardens",43.706397,-79.309937
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
